In [ ]:
""" Integrate the human stuff with the monkey stuff into the same modeling pipeline
"""


In [1]:
%load_ext autoreload
%autoreload 2

from analysis.getModelHumanDists import * 
from analysis.modelAnalyses import *
from analysis.importDrawgood import *
from analysis.compareModelHumanGood import *
from pythonlib.drawmodel.analysis import *


In [3]:
# 1) Extract model parses
ECTRAIN="S12.10.test5"
DAT = loadCheckpoint(trainset=ECTRAIN, loadparse=True, suppressPrint=True, loadbehavior=True)
stimlist = DATgetSolvedStim(DAT, onlyifhasdatflat=True)

Loading dreamcoder checkpoint
Loading dreamcoder tasks
DRAW TASK training set: S12
DO SHAPING: False
training task names:
['S12_13_shaping_0', 'S12_13_shaping_1', 'S12_13_shaping_2', 'S12_13_shaping_3', 'S12_13_shaping_4', 'S12_13_shaping_5', 'S12_13_shaping_6', 'S12_13_shaping_7', 'S12_13_shaping_8', 'S12_13_shaping_9', 'S12_13_shaping_10', 'S12_1', 'S12_5', 'S12_8', 'S12_10', 'S12_32', 'S12_34', 'S12_38', 'S12_20', 'S12_39', 'S12_57', 'S12_79', 'S12_113', 'S12_124', 'S12_126', 'S12_133', 'S12_147', 'S12_155', 'S12_163', 'S12_200', 'S12_214', 'S12_31', 'S12_52', 'S12_55', 'S12_70', 'S12_129', 'S12_139', 'S12_141', 'S12_148', 'S12_207', 'S12_222', 'S12_224', 'S12_229', 'S12_233', 'S12_235', 'S12_243', 'S12_246']
test tasks:
['S12_13_test_1', 'S12_13_test_2', 'S12_13_test_4', 'S12_13_test_5', 'S12_13_test_6', 'S12_13_test_7', 'S12_13_test_8', 'S12_13_test_9', 'S12_13_test_10', 'S12_13_test_11', 'S12_13_test_12', 'S12_132', 'S12_201', 'S12_220', 'S12_247', 'S13_182', 'S13_217', 'S13_219'

In [5]:
# 1) Prune parses, for analysis purposes.
DAT.keys()

dict_keys(['trainset', 'result', 'tasks', 'testtasks', 'programnames', 'programnames_test', 'behaviorexpt', 'savedir', 'parses', 'analysavedir', 'summarysavedir', 'loadparse', 'taskresultdict', 'datall_human', 'savedir_datsegs', 'datflatsavedir', 'savedir_modelhudist', 'savedir_parseanalysis'])

In [16]:
strokes = DAT["parses"][0]["parse"][0]

TypeError: len() takes no keyword arguments

In [19]:
for s in stimlist:
    datflat = DATloadDatFlat(DAT, s)
    print(len(datflat))


16
8
4
8
8
16
16
16
8
16
16
96
32
96
48
32
64
128
16
8
16
16
32
96
32
96
96
576
384
128
192
32
64
1024
1024
512
128
32
256
256
64
256
128
128
96
192
384
10000
3072
1024


In [ ]:
datsegs = DATloadDatSeg(DAT, stimname)
datflat = DATloadDatFlat(DAT, s)
datseg = getSegmentation(datflat, unique_codes=True, dosplits=True, removebadstrokes=True, removeLongVertLine=REMOVELL) 

# 2) Prior: motor efficiency (redo this using continuous parameters?)


# 3) Likelihood: redo this using continouos parametres?

In [ ]:
# human
DAT["datflat_hu"] = getFlatData(DAT["datall_human"])
DAT["datseg_hu"] = getSegmentation(DAT["datflat_hu"], unique_codes=True, dosplits=True, removeLongVertLine=REMOVELL)                                      
# model
datseg = getSegmentation(datflat, unique_codes=True, dosplits=True, removebadstrokes=True, removeLongVertLine=REMOVELL) # get datseg


In [ ]:
# put all into the model object

In [ ]:
## ==== VERSION 2 - LOAD ALL AT ONCE INTO MEMEORY, NOT PIECEMEITL
# 1) Load dreamcoder
# ECTRAINlist = ["S12.10.test4", "S13.10.test4"]
ECTRAINlist = ["S12.10.test5", "S13.10.test5"]
modelkind_list = ["parse", "randomperm"]
ver="aggregate"
use_withplannerscore=True

distances_flat, DAT_all, workerlist, SAVEDIR = loadMultDCHumanDistances(
    ECTRAINlist, modelkind_list, ver, use_withplannerscore)


In [ ]:

## integrate into 


In [ ]:
############# SUMMARIZE HUMAN-HUMAN DISTS

# 1) Collect previously computed human-human distances in a list of dicts (is a full table)
outdict = getHumanHumanDists()

# 2) Collect all distances, meaned over "other humans" grouped by condition.
humanlist = [w["workerID"] for w in workerlist]
stimlist = [t.name for t in DAT_all[0]["testtasks"]]

outdict_summary = []
for human in humanlist:
    print(human)
    cond = extractHumanCond(human, workerlist)
    # - each human gets one value for each cond
    dists_bycond = []
    for cond_other in [0,1]:
        for stim in stimlist:    
            outdict_summary.append({
                "human":human,
                "cond_human":cond,
                "cond_other":cond_other,
                "stim":stim,
                "dist_mean":np.mean([o["dist"] for o in outdict if o["human1"]==human and extractHumanCond(o["human2"], workerlist)==cond_other
                 and o["task"]==stim])
            })
            

In [ ]:
# 3) Get ready for plotting by converting to pandas and aggregating over stim
# i.e. first aggregated over other people, and then over stims - shouldn't matter since is ballanced.

import pandas as pd
from pythonlib.tools import pandastools as pdt

df = pd.DataFrame(outdict_summary)
df = pdt.aggregGeneral(df, ["human", "cond_other", "cond_human"], ["dist_mean"])
df = pdt.applyFunctionToAllRows(df, F=lambda x: x["cond_human"]==x["cond_other"], newcolname="cond_aligned")

# --- get aligned minus notaligned (df --> dict --> df... is dumb).
df_dict = df.to_dict("records")
df_dict_minused = []
for human in humanlist:
    d1 = [d["dist_mean"] for d in df_dict if d["human"]==human and d["cond_aligned"]==True][0]
    d2 = [d["dist_mean"] for d in df_dict if d["human"]==human and d["cond_aligned"]==False][0]
    
    df_dict_minused.append({
        "human":human,
        "dist_aligned_minus_not":d1-d2,
    })
df_aligned = pd.DataFrame(df_dict_minused)
    

In [ ]:
SAVEDIR = f"analysis/summaryfigs/acrossexpt/ecS12.10.test5_S13.10.test5-dg2.4_2.4/closer_analysis/notebook_comparing_human_model_parses"
savedir = f"{SAVEDIR}/human_human_distances"
import os
os.makedirs(savedir, exist_ok=True)

In [ ]:
import seaborn as sns
from pythonlib.tools.snstools import rotateLabel

ax = sns.catplot(x="human", y="dist_mean", hue="cond_other", col="cond_human", data=df, aspect=2, height=4)
rotateLabel(ax)
ax.savefig(f"{savedir}/overview_meanOverTeststimAndOtherhuman_byhuman.pdf")
                                                                
ax = sns.catplot(x="cond_other", y="dist_mean", hue="cond_other", col="cond_human", data=df)
ax.savefig(f"{savedir}/overview_meanOverTeststimAndOtherhuman_aggreg.pdf")

ax = sns.catplot(x="cond_aligned", y="dist_mean", data=df, ci=68, kind="point")
ax.savefig(f"{savedir}/overview_meanbyaligned.pdf")

ax = sns.catplot(y="dist_aligned_minus_not", data=df_aligned, ci=68, kind="point")
plt.ylim([0, -0.04])
ax.savefig(f"{savedir}/overview_grandmean.pdf")





In [ ]:
## SCRATCH
# load model-human distances preprocessed
DATloadModelHuDist(DAT_all[0], "S12_1", "A2VLTSW6CXIUMR", "aggregate", True)

# load processed datsegs (segmented)
DATloadDatSeg(DAT_all[0], "S12_1")[0]

# load stroke data
DATloadDatFlat(DAT_all[0], "S12_1")[0]